(1) 학습/테스트 데이터 셋 분리하지 않고 예측

In [3]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


iris_data = load_iris()
dt_clf = DecisionTreeClassifier()

train_data = iris_data.data
train_label = iris_data.target

# 학습 수행
dt_clf.fit(train_data, train_label)

#테스트
pred=dt_clf.predict(train_data)
print("예측 정확도 : ", accuracy_score(train_label, pred) )

예측 정확도 :  1.0


(2) 학습/테스트 데이터 셋 분리하고 예측

In [5]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

iris_data = load_iris()
dt_clf = DecisionTreeClassifier()

# 분할 (split)
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, test_size=0.3, random_state=20)
print(y_train)

[0 0 0 2 0 2 2 0 2 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 0 0 1 1 1 0 0 2 0 0 1 2 2
 2 2 2 2 1 2 2 2 0 2 0 0 0 2 1 1 2 2 1 0 2 0 1 2 0 1 1 2 0 0 2 2 1 1 1 2 2
 0 2 0 1 0 0 2 2 2 0 0 2 1 1 2 0 2 0 0 2 2 0 0 0 2 1 0 1 2 0 1]


In [9]:
# 학습 수행
dt_clf.fit(X_train, y_train)
# 예측 수행
pred = dt_clf.predict(X_test)
print("예측정확도 : ", accuracy_score(y_test, pred))

예측정확도 :  0.8888888888888888


**넘파이 ndarray 뿐만 아니라 판다스 DataFrame/Series도 train_test_split()으로 분할 가능**

In [11]:
import pandas as pd

iris_df=pd.DataFrame(iris_data.data, columns=iris_data.feature_names)
iris_df['target']=iris_data.target
iris_df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


In [15]:
# 피처 데이터프레임 반환 (마지막 열 전까지, 마지막 열 제외)
feature_df = iris_df.iloc[:, :-1]

# 타깃 데이터 프레임 반환 
target_df = iris_df.iloc[:, -1]

# 학습/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(feature_df, target_df, test_size=0.3, random_state=4)
print(y_train)

13     0
15     0
61     1
91     1
116    2
      ..
50     1
87     1
104    2
129    2
122    2
Name: target, Length: 105, dtype: int64


In [16]:
type(X_train)

pandas.core.frame.DataFrame

In [21]:
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)
perd = dt_clf.predict(X_test)
print('예측정확도:', accuracy_score(y_test, pred))

예측정확도: 0.17777777777777778


**K 폴드 예제**
 

In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

iris = load_iris()
features= iris.data
label = iris.target

features.shape

(150, 4)

In [25]:
# DecisionTreeClassifier 객체 생성
dr_clf = DecisionTreeClassifier(random_state = 156)

# 5개의 폴드 세트로 분리하는 KFold 객체 생성
kfold = KFold(n_splits=5)

# 폴드 세트별 정확도를 담을 리스트 객체 생성
cv_accuaracy = []

In [27]:
# 폴드 별 학습용, 검증용 데이터 세트의 행 인덱스 확인
for train_index, test_index in kfold.split(features):
    print(train_index, test_index)

[ 30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 1

In [40]:
import numpy as np

for train_index, test_index in kfold.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train = label[train_index]
    y_test = label[test_index]
    
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    acc = np.round(accuracy_score(y_test, pred),3)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print("정확도 : %f, 학습데이터크기 : %d, 검증데이터크기 : %d" %(acc, train_size, test_size))
    cv_accuaracy.append(acc)
print("평균 검증 정확도 ", np.mean(cv_accuaracy))


정확도 : 1.000000, 학습데이터크기 : 120, 검증데이터크기 : 30
정확도 : 1.000000, 학습데이터크기 : 120, 검증데이터크기 : 30
정확도 : 0.900000, 학습데이터크기 : 120, 검증데이터크기 : 30
정확도 : 0.933000, 학습데이터크기 : 120, 검증데이터크기 : 30
정확도 : 0.733000, 학습데이터크기 : 120, 검증데이터크기 : 30
평균 검증 정확도  0.9132000000000001


In [41]:
import pandas as pd


iris_df=pd.DataFrame(iris_data.data, columns=iris_data.feature_names)
iris_df['label']=iris_data.target
iris_df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


In [55]:
import pandas as pd

iris = load_iris()
iris_df = pd.DataFrame(data=iris, columns=iris.feature_names)
iris_df['label']=iris.target
iris_df['label'].value_counts()

0    50
1    50
2    50
Name: label, dtype: int64

In [42]:
iris_df['label'].value_counts()
# 레이블 값은 0,1,2 값 모두 50개로 동일
# 즉,setosa, 

0    50
1    50
2    50
Name: label, dtype: int64

In [56]:
# 3개 폴드 구성 

kfold = KFold(n_splits=3)
n=0
for train_index, test_index in kfold.split(iris_df):
    n += 1
    label_train = iris_df['label'].iloc[train_index] 
    label_test = iris_df['label'].iloc[test_index]
    print("[교차검증 : %d]" %(n))
    print("학습용 : ", label_train.value_counts())
    
    print("검증용 : ", label_test.value_counts())
    

[교차검증 : 1]
학습용 :  1    50
2    50
Name: label, dtype: int64
검증용 :  0    50
Name: label, dtype: int64
[교차검증 : 2]
학습용 :  0    50
2    50
Name: label, dtype: int64
검증용 :  1    50
Name: label, dtype: int64
[교차검증 : 3]
학습용 :  0    50
1    50
Name: label, dtype: int64
검증용 :  2    50
Name: label, dtype: int64


In [57]:
# stratifiedKFold 클래스

from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n=0

for trian_index, test_index in skf.split(iris_df, iris_df['label']):
    n=n+1
    label_train = iris_df['label'].iloc[train_index] 
    label_test = iris_df['label'].iloc[test_index]
    print("[교차검증 : {0}]".format(n))
    print("학습용 레이블 분포 : \n ", label_train.value_counts())
    print("검증용 레이블 분포 : \n ", label_test.value_counts())

[교차검증 : 1]
학습용 레이블 분포 : 
  0    50
1    50
Name: label, dtype: int64
검증용 레이블 분포 : 
  0    17
1    17
2    16
Name: label, dtype: int64
[교차검증 : 2]
학습용 레이블 분포 : 
  0    50
1    50
Name: label, dtype: int64
검증용 레이블 분포 : 
  0    17
2    17
1    16
Name: label, dtype: int64
[교차검증 : 3]
학습용 레이블 분포 : 
  0    50
1    50
Name: label, dtype: int64
검증용 레이블 분포 : 
  1    17
2    17
0    16
Name: label, dtype: int64


**붓꽃 자료를 3개 폴드로 분할하여 학습 및 검증**

In [59]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
import numpy as np

iris = load_iris()
dt_clf = DecisionTreeClassifier(random_state = 156)

features = iris.data
label = iris.target

scores = cross_val_score(dt_clf, features, label, scoring = 'accuracy', cv=3)
print('교차 검증별 정확도: ', scores)
print("평균 검증 정확도 : ", np.round(np.mean(scores),4))

교차 검증별 정확도:  [0.98 0.94 0.98]
평균 검증 정확도 :  0.9667


In [71]:
# GridSearchCV를 이용해 결정트리 알고리즘의 여러가지 최적화 파라미터를 순차적으로 적용하여 분석


from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data,
                                                   iris.target,
                                                   test_size=0.2,
                                                   random_state=121)
st_clf = DecisionTreeClassifier()
parameters= {'max_depth' : [1,2,3], 'min_samples_split' : [2,3]}
# 하이퍼파라미터는 딕셔너리 형식으로 지정
# key 값이 결정트리의 하이퍼파라미터
# value : 하이퍼파라미터의 값


In [72]:
grid_tree = GridSearchCV(dt_clf, param_grid=parameters, cv=3, refit=True,
            return_train_score=True)

grid_tree.fit(X_train, y_train)

scores_df = pd.DataFrame(grid_tree.cv_results_)
scores_df 

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.001903,0.000126,0.000427,0.000126,1,2,"{'max_depth': 1, 'min_samples_split': 2}",0.700,0.7,0.70,0.700000,1.110223e-16,5,0.7000,0.7000,0.7000,0.700000,1.110223e-16
1,0.000997,0.000172,0.000403,0.000081,1,3,"{'max_depth': 1, 'min_samples_split': 3}",0.700,0.7,0.70,0.700000,1.110223e-16,5,0.7000,0.7000,0.7000,0.700000,1.110223e-16
2,0.002154,0.000921,0.001375,0.001167,2,2,"{'max_depth': 2, 'min_samples_split': 2}",0.925,1.0,0.95,0.958333,3.118048e-02,3,0.9750,0.9375,0.9625,0.958333,1.559024e-02
3,0.001865,0.000922,0.001283,0.001010,2,3,"{'max_depth': 2, 'min_samples_split': 3}",0.925,1.0,0.95,0.958333,3.118048e-02,3,0.9750,0.9375,0.9625,0.958333,1.559024e-02
4,0.001634,0.000548,0.000684,0.000069,3,2,"{'max_depth': 3, 'min_samples_split': 2}",0.975,1.0,0.95,0.975000,2.041241e-02,1,0.9875,0.9625,0.9875,0.979167,1.178511e-02
5,0.001312,0.000250,0.000973,0.000350,3,3,"{'max_depth': 3, 'min_samples_split': 3}",0.975,1.0,0.95,0.975000,2.041241e-02,1,0.9875,0.9625,0.9875,0.979167,1.178511e-02


In [74]:
# GridSearchCV 결과 세트로 딕셔너리 형태인 cv_results_ 를 df로 변환

scores_df[['params', 'mean_test_score', 'rank_test_score']]


,params,mean_test_score,rank_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1


In [76]:
# 최고 성능을 가지는 파라미터 조합 및 예측 성능 1위 값 출력
print('최적 파라미터 : ',grid_tree.best_params_)
print('최고 정확도 : ', grid_tree.best_score_)


최적 파라미터 :  {'max_depth': 3, 'min_samples_split': 2}
최고 정확도 :  0.975


In [78]:
best_dt = grid_tree.best_estimator_

pred = best_dt.predict(X_test)
accuracy_score(y_test, pred)

0.9666666666666667

In [81]:
# 사이킷런의 레이블 인코딩 클래스 : labelencoder

from sklearn.preprocessing import LabelEncoder

items = ['TV','냉장고','전자렌지','컴퓨터','선풍기','선풍기','믹서','믹서']
encoder = LabelEncoder()
encoder.fit(items)
encoder.transform(items)
labels=encoder.transform(items)
print(labels)

[0 1 4 5 3 3 2 2]


In [82]:
# 인코딩 전 원래의 값 확인 : encoder.classes_ 속성
encoder.classes_

array(['TV', '냉장고', '믹서', '선풍기', '전자렌지', '컴퓨터'], dtype='<U4')

In [84]:
# 인코딩된 값 디코딩
encoder.inverse_transform([3,0,2,1])

array(['선풍기', 'TV', '믹서', '냉장고'], dtype='<U4')

In [86]:
# 원-핫 인코딩(One-Hot encoding)

from sklearn.preprocessing import OneHotEncoder
import numpy as np

items = ['TV','냉장고','전자렌지','컴퓨터','선풍기','선풍기','믹서','믹서']

#1, 숫자값으로 변환을 위해 LabelEncoder로 변환

encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)
labels

array([0, 1, 4, 5, 3, 3, 2, 2])

In [88]:
#2, 2차원 데이터로 변환

labels = labels.reshape(-1,1) 
labels

array([[0],
       [1],
       [4],
       [5],
       [3],
       [3],
       [2],
       [2]])

In [89]:
#3, 원-핫 인코딩을 적용

one_encoder = OneHotEncoder()
one_encoder.fit(labels)
one_labels = one_encoder.transform(labels)
one_labels

<8x6 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>

In [90]:
print(one_labels)

  (0, 0)	1.0
  (1, 1)	1.0
  (2, 4)	1.0
  (3, 5)	1.0
  (4, 3)	1.0
  (5, 3)	1.0
  (6, 2)	1.0
  (7, 2)	1.0


In [91]:
one_labels.toarray()

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]])

In [95]:
# pandat api 사용한 원-핫 인코딩 수행

import pandas as pd

df = pd.DataFrame({'items': ['TV','냉장고','전자렌지','컴퓨터','선풍기','선풍기','믹서','믹서']})


In [98]:
pd.get_dummies(df)

,items_TV,items_냉장고,items_믹서,items_선풍기,items_전자렌지,items_컴퓨터
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,0,0,1,0
3,0,0,0,0,0,1
4,0,0,0,1,0,0
5,0,0,0,1,0,0
6,0,0,1,0,0,0
7,0,0,1,0,0,0


In [99]:
pd.get_dummies(df).to_numpy()

array([[1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0]], dtype=uint8)

In [104]:
# StandardScaler

from sklearn.datasets import load_iris
import pandas as pd
# 불꽃 데이터 셋을 로딩하고 데이터프레임으로 변환


iris = load_iris()
iris_data = iris.data
iris_df = pd.DataFrame(data=iris_data, columns=iris.feature_names)

iris_df.mean() # 평균값
iris_df.var() # 분산값



sepal length (cm)    0.685694
sepal width (cm)     0.189979
petal length (cm)    3.116278
petal width (cm)     0.581006
dtype: float64

In [109]:
# StandardScaler 이용 표준화해서 변환

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

iris_scaled_df = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)

print(iris_scaled_df.mean())
print(iris_scaled_df.std())

sepal length (cm)   -1.690315e-15
sepal width (cm)    -1.842970e-15
petal length (cm)   -1.698641e-15
petal width (cm)    -1.409243e-15
dtype: float64
sepal length (cm)    1.00335
sepal width (cm)     1.00335
petal length (cm)    1.00335
petal width (cm)     1.00335
dtype: float64


In [112]:
# MinMaxScaler

from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler 객체생성
scaler = MinMaxScaler()

# MinMaxScaler 데이터 세트로 변환, fit과 transform 호출
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)
print(iris_scaled)


iris_scaled_df = pd.DataFrame(data=iris_scaled, columns = iris.feature_names)
print(iris_scaled_df.min())
print(iris_scaled_df.max())


[[0.22222222 0.625      0.06779661 0.04166667]
 [0.16666667 0.41666667 0.06779661 0.04166667]
 [0.11111111 0.5        0.05084746 0.04166667]
 [0.08333333 0.45833333 0.08474576 0.04166667]
 [0.19444444 0.66666667 0.06779661 0.04166667]
 [0.30555556 0.79166667 0.11864407 0.125     ]
 [0.08333333 0.58333333 0.06779661 0.08333333]
 [0.19444444 0.58333333 0.08474576 0.04166667]
 [0.02777778 0.375      0.06779661 0.04166667]
 [0.16666667 0.45833333 0.08474576 0.        ]
 [0.30555556 0.70833333 0.08474576 0.04166667]
 [0.13888889 0.58333333 0.10169492 0.04166667]
 [0.13888889 0.41666667 0.06779661 0.        ]
 [0.         0.41666667 0.01694915 0.        ]
 [0.41666667 0.83333333 0.03389831 0.04166667]
 [0.38888889 1.         0.08474576 0.125     ]
 [0.30555556 0.79166667 0.05084746 0.125     ]
 [0.22222222 0.625      0.06779661 0.08333333]
 [0.38888889 0.75       0.11864407 0.08333333]
 [0.22222222 0.75       0.08474576 0.08333333]
 [0.30555556 0.58333333 0.11864407 0.04166667]
 [0.22222222 